# Section 2: Building a Wallet Portfolio Manager

In this section, you'll learn to:
- Calculate current wallet net worth
- Build portfolio performance charts
- Analyze individual token holdings

## APIs Used in This Section

### REST API Endpoints:
- **`GET /wallet/v2/current-net-worth`** - Get current wallet portfolio value and token breakdown
- **`GET /wallet/v2/net-worth`** - Retrieve historical net worth data for performance tracking
- **`GET /wallet/v2/net-worth-details`** - Get detailed breakdown of a wallet token holdings and values at any point of time

Let's build a professional-grade portfolio management system!

## Setup and Imports

**Important**: This section requires a **Business API Key** from Birdeye Data Services.

### Getting Your Business API Key

The wallet portfolio features require access to endpoints that are only available with a Business package. You will be provided with a Business API key.

**Add the Business API key to your .env file:**
```
BDS_API_KEY=your_business_api_key_here
```

If you don't have a Business API key, you can:
1. Contact workshop host for a temporary key
2. Sign up for a Business plan at [Birdeye Data Services](https://bds.birdeye.so)
3. Continue with the demo to see how the features work

In [ ]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, HTML, Markdown, clear_output
import ipywidgets as widgets
from datetime import datetime, timedelta
import json
import time
import threading
import warnings
warnings.filterwarnings('ignore')

# Force reload of utils module to ensure latest version
import utils
import importlib
importlib.reload(utils)

# Import our custom utilities
from utils import (
    BirdeyeDataServices, 
    BirdeyeDataServicesWebSocket,
    create_portfolio_chart,
    create_portfolio_pie_chart,
    create_portfolio_history_pie_chart,
    create_candlestick_chart,
    format_currency,
    format_transaction_data,
    check_api_key
)

print("All modules imported successfully, including create_portfolio_pie_chart!")

# Check Business API key and initialize client
if check_api_key('business'):
    # Initialize Birdeye Data Services client with Business API key
    birdeye = BirdeyeDataServices(api_key_type='business')
    print("Setup complete! Ready to analyze portfolios with Business API.")
else:
    print("Please configure BDS_API_KEY in your .env file")
    print("Contact your workshop host for a Business API key")

### Troubleshooting Note

**If you encounter import errors for `create_portfolio_pie_chart`:**
1. **Restart the kernel**: Go to `Kernel` → `Restart` in the menu
2. **Re-run the import cell above**
3. **Clear browser cache** if the issue persists

The function exists in `utils.py` and the reload code above should resolve any caching issues.

## Step 1: Set Up Wallet Address

Let's configure a wallet address for portfolio analysis:

In [ ]:
# Example wallet addresses (you can replace with any Solana wallet address)
EXAMPLE_WALLETS = {
    "Example Wallet 1": "4Be9CvxqHW6BYiRAxW9Q3xu1ycTMWaL5z8NX4HR3ha7t",
    "Example Wallet 2": "h9QLBKdx5VbPFoEUc8pfejqTH5PhCVo3ewwrrWiikGb",
    "Example Wallet 3": "6qBogbNZYGvzSb1UfGU7BPggDgFbb34DEr3fmCkFaf8T"
}

# Create wallet selector
wallet_dropdown = widgets.Dropdown(
    options=[(name, addr) for name, addr in EXAMPLE_WALLETS.items()],
    value=list(EXAMPLE_WALLETS.values())[0],
    description='Select Wallet:',
    style={'description_width': 'initial'}
)

custom_wallet_input = widgets.Text(
    value='',
    placeholder='Or enter custom wallet address',
    description='Custom Wallet:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

def get_selected_wallet():
    if custom_wallet_input.value.strip():
        return custom_wallet_input.value.strip()
    return wallet_dropdown.value

print("Wallet Configuration:")
display(widgets.VBox([wallet_dropdown, custom_wallet_input]))

# Set initial wallet
current_wallet = get_selected_wallet()
print(f"\nCurrent wallet: {current_wallet}")

## Step 2: Get Current Net Worth

Let's fetch the current net worth of the selected wallet:

In [ ]:
# Step 2: Get Current Net Worth - Interactive Version
def get_current_net_worth(wallet_address=None):
    if not wallet_address:
        wallet_address = get_selected_wallet()
    
    print(f"Fetching current net worth for wallet: {wallet_address}")
    
    net_worth_data = birdeye.get_wallet_net_worth(wallet_address)
    
    if net_worth_data and 'data' in net_worth_data:
        data = net_worth_data['data']
        
        try:
            total_usd = float(data.get('total_value', 0))
        except (ValueError, TypeError):
            total_usd = 0.0
        
        print(f"\nCurrent Portfolio Value: {format_currency(total_usd)}")
        print("-" * 50)
        
        if 'items' in data and data['items']:
            items = data['items']
            print(f"Portfolio Breakdown ({len(items)} tokens):")
            
            for item in items[:10]:
                symbol = item.get('symbol', 'Unknown')
                
                try:
                    value_usd = float(item.get('value', 0))
                except (ValueError, TypeError):
                    value_usd = 0.0
                
                try:
                    balance = int(float(item.get('balance', 0)))
                except (ValueError, TypeError):
                    balance = 0
                
                percentage = (value_usd / total_usd * 100) if total_usd > 0 else 0
                
                print(f"   • {symbol}: {format_currency(value_usd)} ({percentage:.1f}%) - {balance:.4f} tokens")
            
            print("\nCreating portfolio allocation pie chart...")
            pie_chart = create_portfolio_pie_chart(net_worth_data, f"Portfolio Allocation - {wallet_address[:8]}...")
            
            if pie_chart:
                pie_chart.show()
            else:
                print("Could not create pie chart")
        else:
            print("No individual token holdings found")
        
        print("\nNet worth data loaded successfully!")
        return total_usd
    else:
        print("Could not fetch net worth data. The wallet might be empty or invalid.")
        return 0

# Interactive controls for Step 2
step2_checkbox = widgets.Checkbox(
    value=False,
    description='Use custom wallet address',
    style={'description_width': 'initial'}
)

step2_wallet_input = widgets.Text(
    value='',
    placeholder='Enter wallet address',
    description='Wallet:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

step2_button = widgets.Button(
    description='Get Net Worth',
    button_style='primary',
    tooltip='Fetch current net worth'
)

step2_output = widgets.Output()

def on_step2_button_click(b):
    with step2_output:
        clear_output(wait=True)
        if step2_checkbox.value and step2_wallet_input.value.strip():
            get_current_net_worth(step2_wallet_input.value.strip())
        else:
            get_current_net_worth()

step2_button.on_click(on_step2_button_click)

# Initial execution
get_current_net_worth()

# Display interactive controls
print("\n" + "="*60)
print("Interactive Controls:")
display(widgets.VBox([
    step2_checkbox,
    step2_wallet_input,
    step2_button,
    step2_output
]))

## Step 3: Portfolio Performance Chart

Let's visualize the portfolio's performance over time:

In [ ]:
# Step 3: Portfolio Performance Chart - Interactive Version
def get_portfolio_history(wallet_address=None):
    if not wallet_address:
        wallet_address = get_selected_wallet()
    
    print(f"Fetching portfolio history for wallet: {wallet_address}")
    net_worth_history = birdeye.get_wallet_net_worth_history(wallet_address)
    
    if net_worth_history and 'data' in net_worth_history:
        portfolio_chart = create_portfolio_chart(net_worth_history)
        
        if portfolio_chart:
            print("\nPortfolio Performance Over Time (Line Chart):")
            portfolio_chart.show()
            
            history_data = net_worth_history['data']['history']
            if len(history_data) > 1:
                try:
                    first_value = float(history_data[0].get('net_worth', 0))
                    last_value = float(history_data[-1].get('net_worth', 0))
                except (ValueError, TypeError):
                    first_value = last_value = 0.0
                
                if first_value > 0:
                    print(f"\nPerformance Metrics:")
                    print(f"   Value Change: {format_currency(last_value - first_value)}")
                    
                    values = []
                    for item in history_data:
                        try:
                            val = float(item.get('net_worth', 0))
                            values.append(val)
                        except (ValueError, TypeError):
                            values.append(0.0)
                    
                    if values:
                        max_value = max(values)
                        min_value = min(values)
                        
                        print(f"   High: {format_currency(max_value)}")
                        print(f"   Low: {format_currency(min_value)}")
        else:
            print("Could not create portfolio line chart")
    else:
        print("Could not fetch portfolio history")

# Interactive controls for Step 3
step3_checkbox = widgets.Checkbox(
    value=False,
    description='Use custom wallet address',
    style={'description_width': 'initial'}
)

step3_wallet_input = widgets.Text(
    value='',
    placeholder='Enter wallet address',
    description='Wallet:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

step3_button = widgets.Button(
    description='Get Portfolio History',
    button_style='info',
    tooltip='Fetch portfolio performance chart'
)

step3_output = widgets.Output()

def on_step3_button_click(b):
    with step3_output:
        clear_output(wait=True)
        if step3_checkbox.value and step3_wallet_input.value.strip():
            get_portfolio_history(step3_wallet_input.value.strip())
        else:
            get_portfolio_history()

step3_button.on_click(on_step3_button_click)

# Initial execution
get_portfolio_history()

# Display interactive controls
print("\n" + "="*60)
print("Interactive Controls:")
display(widgets.VBox([
    step3_checkbox,
    step3_wallet_input,
    step3_button,
    step3_output
]))

## Step 4: Interactive Portfolio Analyzer

Let's create an interactive tool to analyze any wallet's portfolio with custom time selection and pie chart visualization:

In [ ]:
# Interactive Portfolio Analyzer
class PortfolioAnalyzer:
    def __init__(self):
        self.wallet_input = widgets.Text(
            value='9WzDXwBbmkg8ZTbNMqUxvQRAyrZzDsGYdLVL9zYtAWWM',
            placeholder='Enter wallet address',
            description='Wallet Address:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='500px')
        )
        
        self.time_select = widgets.Dropdown(
            options=[
                ('Current Time', 'current'),
                ('Custom Date', 'custom')
            ],
            value='current',
            description='Time Selection:',
            style={'description_width': 'initial'}
        )
        
        self.date_picker = widgets.DatePicker(
            description='Select Date:',
            disabled=True,
            style={'description_width': 'initial'}
        )
        
        self.analyze_button = widgets.Button(
            description='Analyze Portfolio',
            button_style='primary',
            tooltip='Analyze the wallet portfolio'
        )
        
        self.output = widgets.Output()
        
        # Set up event handlers
        self.time_select.observe(self.on_time_selection_change, names='value')
        self.analyze_button.on_click(self.analyze_portfolio)
    
    def on_time_selection_change(self, change):
        if change['new'] == 'custom':
            self.date_picker.disabled = False
        else:
            self.date_picker.disabled = True
    
    def analyze_portfolio(self, button):
        with self.output:
            clear_output(wait=True)
            
            wallet_address = self.wallet_input.value.strip()
            time_selection = self.time_select.value
            
            if not wallet_address:
                print("Please enter a wallet address")
                return

            print(f"Analyzing portfolio for: {wallet_address}")
            
            if time_selection == 'custom' and self.date_picker.value:
                selected_date = self.date_picker.value
                print(f"Custom date: {selected_date}")
                # Convert date to formatted string for API call (format: '2025-07-13 02:00:00')
                time_str = datetime.combine(selected_date, datetime.min.time()).strftime('%Y-%m-%d %H:%M:%S')
                print(f"Using time format: {time_str} for net-worth-details API")
            
            print("\n" + "="*50)
            
            # Get portfolio data
            if time_selection == 'current':
                # Use current net-worth API
                net_worth_data = birdeye.get_wallet_net_worth_details(wallet_address)
            elif time_selection == 'custom':
                # Use custom date string for net-worth-details API
                net_worth_data = birdeye.get_wallet_net_worth_details(wallet_address, time=time_str)
            else:
                net_worth_data = birdeye.get_wallet_net_worth(wallet_address)
            
            if net_worth_data and 'data' in net_worth_data:
                data = net_worth_data['data']
                
                # Coerce total_value to float with error handling
                try:
                    total_value = float(data.get('net_worth', 0))
                except (ValueError, TypeError):
                    total_value = 0.0
                
                print(f"Total Portfolio Value: {format_currency(total_value)}")
                
                if 'net_assets' in data and data['net_assets']:
                    items = data['net_assets']
                    print(f"Number of tokens: {len(items)}")
                    print("\nTop Holdings:")
                    
                    for i, item in enumerate(items[:20], 1):
                        symbol = item.get('symbol', 'Unknown')
                        
                        # Coerce value to float with error handling
                        try:
                            value = float(item.get('value', 0))
                        except (ValueError, TypeError):
                            value = 0.0
                        
                        # Coerce balance to float with error handling
                        try:
                            balance = float(item.get('balance', 0))
                        except (ValueError, TypeError):
                            balance = 0.0
                        
                        percentage = (value / total_value * 100) if total_value > 0 else 0
                        
                        print(f"   {i}. {symbol}: {format_currency(value)} ({percentage:.1f}%)")
                        print(f"      Balance: {balance:.4f} tokens")
                    
                    # Create pie chart
                    print("\nCreating portfolio pie chart...")
                    pie_chart = create_portfolio_history_pie_chart(net_worth_data, f"Portfolio - {wallet_address}")
                    
                    if pie_chart:
                        pie_chart.show()
                    else:
                        print("Could not create pie chart")
                else:
                    print("No token holdings found")
            else:
                print("Could not fetch portfolio data")
    
    def get_widget(self):
        return widgets.VBox([
            self.wallet_input,
            self.time_select,
            self.date_picker,
            self.analyze_button,
            self.output
        ])

# Create and display the analyzer
analyzer = PortfolioAnalyzer()
display(analyzer.get_widget())

## Congratulations!

You've successfully built a comprehensive wallet portfolio management system! Here's what you've accomplished:

### What You've Built:
- **Wallet Configuration**: Easy wallet address management and selection
- **Current Net Worth Analysis**: Real-time portfolio valuation with token breakdown
- **Portfolio Performance Tracking**: Historical net worth charts with performance metrics
- **Interactive Portfolio Analyzer**: Custom time-based analysis with pie chart visualization

### Additional Resources:
- [Birdeye API Documentation](https://docs.birdeye.so)